In [187]:
import pandas as pd
import numpy as np
import json
import tqdm

In [188]:
game_ids_df = pd.read_excel("../data/game_ids.xlsx")
game_ids_df = game_ids_df.astype({"tournamentId":"str", "matchId":"str", "gameId":"str"})
game_ids_df.drop("Unnamed: 0", axis=1, inplace=True)
game_ids_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9287 entries, 0 to 9286
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   startTime(match)    9287 non-null   object 
 1   tournamentId        9287 non-null   object 
 2   blockName           9287 non-null   object 
 3   leagueName          9287 non-null   object 
 4   matchId             9287 non-null   object 
 5   bestof              9287 non-null   int64  
 6   blueteam_name       9287 non-null   object 
 7   blueteam_code       9287 non-null   object 
 8   blueteam_win        9287 non-null   int64  
 9   redteam_name        9287 non-null   object 
 10  redteam_code        9287 non-null   object 
 11  redteam_win         9287 non-null   int64  
 12  gameNumberInAMatch  9287 non-null   int64  
 13  gameId              9287 non-null   object 
 14  patch               8081 non-null   float64
 15  pick_0              8023 non-null   object 
 16  pick_1

In [189]:
game_ids_with_picks_df = game_ids_df[pd.isna(game_ids_df["pick_0"]) == False].copy()
game_ids_with_picks_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8023 entries, 0 to 9286
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   startTime(match)    8023 non-null   object 
 1   tournamentId        8023 non-null   object 
 2   blockName           8023 non-null   object 
 3   leagueName          8023 non-null   object 
 4   matchId             8023 non-null   object 
 5   bestof              8023 non-null   int64  
 6   blueteam_name       8023 non-null   object 
 7   blueteam_code       8023 non-null   object 
 8   blueteam_win        8023 non-null   int64  
 9   redteam_name        8023 non-null   object 
 10  redteam_code        8023 non-null   object 
 11  redteam_win         8023 non-null   int64  
 12  gameNumberInAMatch  8023 non-null   int64  
 13  gameId              8023 non-null   object 
 14  patch               8023 non-null   float64
 15  pick_0              8023 non-null   object 
 16  pick_1     

In [190]:
crawled_df = pd.read_excel("../data/crawling_result_with_codenames.xlsx")
crawled_df = crawled_df.astype({"tournament_id":"str", "patch":"str"})
crawled_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20179 entries, 0 to 20178
Data columns (total 31 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   date              20179 non-null  object
 1   tournament_title  20179 non-null  object
 2   patch             20179 non-null  object
 3   blueteam          20179 non-null  object
 4   redteam           20179 non-null  object
 5   winner_side       20179 non-null  object
 6   ban_0             20135 non-null  object
 7   ban_1             20152 non-null  object
 8   ban_2             20147 non-null  object
 9   ban_3             16973 non-null  object
 10  ban_4             16887 non-null  object
 11  ban_5             20127 non-null  object
 12  ban_6             20151 non-null  object
 13  ban_7             20143 non-null  object
 14  ban_8             16978 non-null  object
 15  ban_9             16875 non-null  object
 16  pick_0            20179 non-null  object
 17  pick_1      

In [191]:
pick_column_list = [f"pick_{x}" for x in range(10)]
left_on = ["tournamentId", "blueteam_code", "redteam_code"]
left_on_reversed = ["tournamentId", "redteam_code", "blueteam_code"]
right_on = ["tournament_id", "blueteam_code", "redteam_code"]
left_on.extend(pick_column_list)
left_on_reversed.extend(pick_column_list)
right_on.extend(pick_column_list)
game_ids_with_picks_df = game_ids_df.astype({"patch":"str"})
print(left_on)
print(left_on_reversed)
print(right_on)




['tournamentId', 'blueteam_code', 'redteam_code', 'pick_0', 'pick_1', 'pick_2', 'pick_3', 'pick_4', 'pick_5', 'pick_6', 'pick_7', 'pick_8', 'pick_9']
['tournamentId', 'redteam_code', 'blueteam_code', 'pick_0', 'pick_1', 'pick_2', 'pick_3', 'pick_4', 'pick_5', 'pick_6', 'pick_7', 'pick_8', 'pick_9']
['tournament_id', 'blueteam_code', 'redteam_code', 'pick_0', 'pick_1', 'pick_2', 'pick_3', 'pick_4', 'pick_5', 'pick_6', 'pick_7', 'pick_8', 'pick_9']


In [192]:
game_ids_with_picks_df[left_on].head()

,tournamentId,blueteam_code,redteam_code,pick_0,pick_1,pick_2,pick_3,pick_4,pick_5,pick_6,pick_7,pick_8,pick_9
0,112099354464733424,FLY,PSG,Urgot,Sejuani,Taliyah,Jinx,Nautilus,Rumble,JarvanIV,Azir,Senna,TahmKench
1,112099354464733424,FLY,PSG,Urgot,Maokai,Taliyah,Lucian,Nami,KSante,Sejuani,Azir,Kalista,Ashe
2,112099354464733424,FLY,PSG,KSante,XinZhao,Karma,Kalista,Rell,Olaf,Sejuani,Orianna,Varus,Nautilus
3,112099354464733424,T1,EST,KSante,LeeSin,Orianna,Kalista,Neeko,Yone,Volibear,Ahri,Draven,Rell
4,112099354464733424,T1,EST,Aatrox,LeeSin,Ahri,Draven,Ashe,KSante,Viego,Orianna,Kalista,Rumble


In [193]:
crawled_df[right_on].head()

,tournament_id,blueteam_code,redteam_code,pick_0,pick_1,pick_2,pick_3,pick_4,pick_5,pick_6,pick_7,pick_8,pick_9
0,112099354464733424,GEN,BLG,Rumble,Lee Sin,Azir,Senna,Maokai,Camille,Nidalee,Tristana,Zeri,Nautilus
1,112099354464733424,BLG,GEN,Jax,Nidalee,Jayce,Senna,Tahm Kench,Rumble,Lee Sin,Hwei,Kalista,Ashe
2,112099354464733424,BLG,GEN,Camille,Xin Zhao,Orianna,Varus,Ashe,Zac,Sejuani,Yone,Kalista,Blitzcrank
3,112099354464733424,GEN,BLG,K'Sante,Karthus,Yone,Senna,Tahm Kench,Twisted Fate,Xin Zhao,Neeko,Lucian,Nami
4,112099354464733424,BLG,T1,Camille,Xin Zhao,Neeko,Senna,Ornn,K'Sante,Viego,Taliyah,Varus,Nautilus


In [194]:
picks_of_game_ids_set = set(np.concatenate(game_ids_with_picks_df.loc[:,pick_column_list].values.tolist()).tolist())
len(picks_of_game_ids_set)

165

In [195]:
picks_of_crawled_set = set(np.concatenate(crawled_df.loc[:,pick_column_list].values.tolist()).tolist())
len(picks_of_crawled_set)

165

In [196]:
difference_of_game_ids = picks_of_game_ids_set.difference(picks_of_crawled_set)
difference_of_crawled = picks_of_crawled_set.difference(picks_of_game_ids_set)
print(difference_of_game_ids)
print(len(difference_of_game_ids))
print(difference_of_crawled)
print(len(difference_of_crawled))



{'KogMaw', 'MonkeyKing', 'KSante', 'TahmKench', 'Renata', 'LeeSin', 'Chogath', 'Belveth', 'RekSai', 'JarvanIV', 'TwistedFate', 'Khazix', 'XinZhao', 'DrMundo', 'nan', 'AurelionSol', 'MissFortune', 'Velkoz', 'Leblanc', 'Kaisa', 'FiddleSticks'}
21
{"Kha'Zix", 'Jarvan IV', "Rek'Sai", "Vel'Koz", 'Twisted Fate', "Bel'Veth", "Kog'Maw", 'Miss Fortune', 'Master Yi', "Cho'Gath", 'Aurelion Sol', 'Xin Zhao', 'Renata Glasc', 'Wukong', 'Dr. Mundo', 'Fiddlesticks', "K'Sante", "Kai'Sa", 'Tahm Kench', 'Lee Sin', 'LeBlanc'}
21


In [197]:
with open('../data/pick_name_not_the_same.json', 'r') as f:
    pick_difference_map = json.load(f)
crawled_df[pick_column_list] = crawled_df[pick_column_list].replace(pick_difference_map)

In [198]:
picks_of_game_ids_set = set(np.concatenate(game_ids_with_picks_df.loc[:,pick_column_list].values.tolist()).tolist())
picks_of_crawled_set = set(np.concatenate(crawled_df.loc[:,pick_column_list].values.tolist()).tolist())
difference_of_game_ids = picks_of_game_ids_set.difference(picks_of_crawled_set)
difference_of_crawled = picks_of_crawled_set.difference(picks_of_game_ids_set)
print(difference_of_game_ids)
print(len(difference_of_game_ids))
print(difference_of_crawled)
print(len(difference_of_crawled))


{'nan'}
1
{'Master Yi'}
1


In [199]:
result_df = pd.merge(game_ids_with_picks_df, crawled_df, how = "left", left_on = left_on, right_on = right_on)
result_df_reversed = pd.merge(game_ids_with_picks_df, crawled_df, how = "left", left_on = left_on_reversed, right_on = right_on)

In [200]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9306 entries, 0 to 9305
Data columns (total 44 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   startTime(match)    9306 non-null   object
 1   tournamentId        9306 non-null   object
 2   blockName           9306 non-null   object
 3   leagueName          9306 non-null   object
 4   matchId             9306 non-null   object
 5   bestof              9306 non-null   int64 
 6   blueteam_name       9306 non-null   object
 7   blueteam_code       9306 non-null   object
 8   blueteam_win        9306 non-null   int64 
 9   redteam_name        9306 non-null   object
 10  redteam_code        9306 non-null   object
 11  redteam_win         9306 non-null   int64 
 12  gameNumberInAMatch  9306 non-null   int64 
 13  gameId              9306 non-null   object
 14  patch_x             9306 non-null   object
 15  pick_0              8042 non-null   object
 16  pick_1              8042

In [201]:
result_df_reversed.rename(columns = {"blueteam_code_x":"blueteam_code", "redteam_code_x":"redteam_code"}, inplace=True)
result_df_reversed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9296 entries, 0 to 9295
Data columns (total 46 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   startTime(match)    9296 non-null   object
 1   tournamentId        9296 non-null   object
 2   blockName           9296 non-null   object
 3   leagueName          9296 non-null   object
 4   matchId             9296 non-null   object
 5   bestof              9296 non-null   int64 
 6   blueteam_name       9296 non-null   object
 7   blueteam_code       9296 non-null   object
 8   blueteam_win        9296 non-null   int64 
 9   redteam_name        9296 non-null   object
 10  redteam_code        9296 non-null   object
 11  redteam_win         9296 non-null   int64 
 12  gameNumberInAMatch  9296 non-null   int64 
 13  gameId              9296 non-null   object
 14  patch_x             9296 non-null   object
 15  pick_0              8032 non-null   object
 16  pick_1              8032

In [202]:
result_df["blueteam_code_y"] = np.nan
result_df["redteam_code_y"] = np.nan
new_df = pd.merge(result_df, result_df_reversed, how="inner", on = left_on)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11735 entries, 0 to 11734
Data columns (total 79 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   startTime(match)_x    11735 non-null  object 
 1   tournamentId          11735 non-null  object 
 2   blockName_x           11735 non-null  object 
 3   leagueName_x          11735 non-null  object 
 4   matchId_x             11735 non-null  object 
 5   bestof_x              11735 non-null  int64  
 6   blueteam_name_x       11735 non-null  object 
 7   blueteam_code         11735 non-null  object 
 8   blueteam_win_x        11735 non-null  int64  
 9   redteam_name_x        11735 non-null  object 
 10  redteam_code          11735 non-null  object 
 11  redteam_win_x         11735 non-null  int64  
 12  gameNumberInAMatch_x  11735 non-null  int64  
 13  gameId_x              11735 non-null  object 
 14  patch_x_x             11735 non-null  object 
 15  pick_0             

In [203]:
result_df.to_excel("../data/game_ids_with_result.xlsx", index=None)
result_df_reversed.to_excel("../data/game_ids_with_result_reversed.xlsx", index = None)

In [204]:
new_df.to_excel("../data/game_ids_with_result_test.xlsx", index = True)